In [379]:
import pandas as pd

# Load cleaned ratings data
ratings = pd.read_csv('../data/cleaned_remapped_ratings.csv')
movies = pd.read_csv('../data/cleaned_remapped_movies.csv')

print("Ratings Shape:", ratings.shape)
print("Movies Shape:", movies.shape)

num_users = ratings['userId'].nunique()  # Unique users
num_movies = ratings['movieId'].nunique()  # Unique movies
print("Shape of user-item interaction matrix:", (num_users, num_movies))

Ratings Shape: (31842705, 4)
Movies Shape: (87382, 6)
Shape of user-item interaction matrix: (200948, 31961)


In [381]:
from scipy.sparse import csr_matrix

# Create the sparse matrix after ensuring movieId is sequential
user_movie_matrix = csr_matrix(
    (ratings['rating'], (ratings['userId'], ratings['movieId']))
)

print("User-Movie Interaction Matrix Shape:", user_movie_matrix.shape)

User-Movie Interaction Matrix Shape: (200949, 31961)


In [383]:
import pickle
from sklearn.decomposition import TruncatedSVD
import numpy as np

# Perform SVD
svd = TruncatedSVD(n_components=50, random_state=42)  # 50 latent factors

user_factors = svd.fit_transform(user_movie_matrix)
# Save user factors
np.save("../models/user_factors.npy", user_factors)
print("Saved user latent factors to '../models/user_factors.npy'.")
print("User factors shape:", user_factors.shape)


movie_factors = svd.components_.T
# Save movie factors
np.save("../models/movie_factors.npy", movie_factors)
print("Saved movie latent factors to '../models/movie_factors.npy'.")
print("Movie factors shape: ", user_factors.shape)


# Save the SVD model to a file
with open("../models/svd_model.pkl", "wb") as f:
    pickle.dump(svd, f)

print("SVD model saved as '../models/svd_model.pkl'.")

Saved user latent factors to '../models/user_factors.npy'.
User factors shape: (200949, 50)
Saved movie latent factors to '../models/movie_factors.npy'.
Movie factors shape:  (200949, 50)
SVD model saved as '../models/svd_model.pkl'.


In [385]:
def recommend_collaborative(user_id, num_recommendations=10):
    # Get user vector
    user_vector = user_factors[user_id]  # Assuming user IDs start at 1
    # user_vector = user_factors[user_id - 1]  # Assuming user IDs start at 1

    # Compute scores for all movies
    scores = np.dot(movie_factors, user_vector)

    # Get top movie indices
    recommended_movie_indices = np.argsort(scores)[::-1][:num_recommendations]

    # Map indices back to movie IDs
    recommended_movie_ids = [movies.iloc[i]['movieId'] for i in recommended_movie_indices]
    return movies[movies['movieId'].isin(recommended_movie_ids)]['title']

In [387]:
# Test recommend_collaborative 1

# Load user factors (NumPy array)
user_factors = np.load("../models/user_factors.npy")

# Load movie factors (NumPy array)
movie_factors = np.load("../models/movie_factors.npy")

# Load movie_indices from a .pkl file
with open('../models/movie_indices.pkl', 'rb') as f:
    movie_indices = pickle.load(f)
# movie_indices = {k.lower(): v for k, v in movie_indices.items()}  # Normalize keys

# # Print the first 10 movie titles and their indices
# for i, (title, index) in enumerate(movie_indices.items()):
#     if i >= 10:  # Stop after printing the first 10
#         break
#     print(f"Title: {title}, Index: {index}")

print(":::::::: Files Loaded ....... ")

# print("Movies Shape:", movies.shape)

:::::::: Files Loaded ....... 


In [389]:
# Test recommend_collaborative 2
# Test the function with a specific user ID
user_id = 3  # Replace with a valid user ID
num_recommendations = 10  # Adjust the number of recommendations as needed

# Generate recommendations
recommended_movies = recommend_collaborative(user_id, num_recommendations)

# Display recommendations
print("Recommended Movies:")
print(recommended_movies)

Recommended Movies:
8                                    sudden death (1995)
15                                         casino (1995)
21                                        copycat (1995)
22                                      assassins (1995)
60                                 eye for an eye (1996)
161                                     desperado (1995)
173                                          kids (1995)
197    umbrellas of cherbourg, the (parapluies de che...
209                         browning version, the (1994)
212                before the rain (pred dozhdot) (1994)
Name: title, dtype: object


In [295]:
import h5py

def find_similar_movies(file_name, target_index, movie_indices, top_n=10):
    """
    Finds the top N similar movies to a target movie using cosine similarity.
    Args:
        file_name (str): Path to the HDF5 file containing cosine similarities.
        target_index (int): Index of the target movie.
        movie_indices (pd.Series): Mapping of movie titles to indices.
        top_n (int): Number of similar movies to return.
    Returns:
        list: List of tuples containing similar movie titles and their similarity scores.
    """
    
    with h5py.File(file_name, 'r') as f:
        # Access the full row for the target movie
        similarities = None
        # print(":::::: Length keys => ", len(f.keys()))
        for batch_name in f.keys():
            # print("Searching batch name : ", batch_name)
            dataset = f[batch_name]
            # print(f":::::: Target Index => {target_index}, Shape => {dataset.shape[0]}")
            if target_index < dataset.shape[0]:  # Check if the index is within this batch
                similarities = dataset[target_index]
                break

        if similarities is None:
            raise ValueError(f"Index {target_index} not found in HDF5 file.")

    # Get the top N similar movies (excluding itself)
    similar_indices = np.argsort(-similarities)  # Sort in descending order
    similar_indices = [i for i in similar_indices if i != target_index][:top_n]

    # Map indices back to movie titles
    index_to_title = {v: k for k, v in movie_indices.items()}
    recommendations = [(index_to_title[idx], similarities[idx]) for idx in similar_indices]

    return recommendations

In [375]:
def recommend_hybrid(title, user_id, num_recommendations=10, content_weight=0.5, collab_weight=0.5):
    # Validate and normalize input
    # normalized_title = title.strip().lower()
    normalized_title = title
    # print(":::::::", normalized_title)
    # normalized_title = title
    # movie_indices = {k.lower(): v for k, v in movie_indices.items()}  # Normalize keys
    if normalized_title not in movie_indices:
        raise ValueError(f"Movie title '{title}' not found.")
    if user_id is not None and (user_id < 1 or user_id > len(user_factors)):
        raise ValueError(f"Invalid user_id: {user_id}.")

    # Content-based recommendations
    content_indices = movie_indices[normalized_title]
    match_movie_tuples = find_similar_movies(
        file_name='../models/cosine_sim.h5',
        target_index=content_indices,
        movie_indices=movie_indices,
        top_n=10
    )

    print("::::::: Match movie tuples before : ")
    for tup in match_movie_tuples:
        print(f":::: {tup}")
    print("\n\n")
    print("Movies length => ", len(movies))
    duplicate_titles = movies[movies.duplicated(subset='title', keep=False)]
    print("Duplicate Titles:\n", len(duplicate_titles))
    print("\n\n")
    # Map movie titles in match_movie_tuples to indices
    title_to_index = {v: k for k, v in movies['title'].to_dict().items()}

    print(":::::::::: Debug valid match movie tuples : ")
    for tup in match_movie_tuples :
        if tup[0] in title_to_index:
            print(tup)
    print("\n\n")
    
    print("::::::: Title to index => ", len(title_to_index))
    valid_match_movie_tuples = [
        (title_to_index.get(tup[0], None), tup[1]) for tup in match_movie_tuples if tup[0] in title_to_index
    ]

    print("::::::: Valid match movie tuples after => ", len(valid_match_movie_tuples))
    for tup in valid_match_movie_tuples:
        print(f":::: {tup}")
    print("\n\n")

    # Filter out invalid matches
    valid_match_movie_tuples = [(idx, score) for idx, score in valid_match_movie_tuples if idx is not None]

    # Collaborative recommendations (only if user_id is provided)
    if user_id is not None:
        user_vector = user_factors[user_id - 1]
        collab_scores = np.dot(movie_factors, user_vector)
    else:
        collab_scores = np.zeros(len(movie_factors))  # No collaborative scores

    # Convert content scores to full array
    content_scores_full = np.zeros(len(collab_scores))
    for idx, score in valid_match_movie_tuples:
        if idx < len(collab_scores):  # Ensure valid index
            content_scores_full[idx] = score

    # Combine scores
    combined_scores = content_weight * content_scores_full + collab_weight * collab_scores
    print(":::::: Length of collab weight => ", collab_weight)
    print(":::::: Collab weight  => ", collab_weight)

    # Get top movie indices
    recommended_movie_indices = np.argsort(combined_scores)[::-1][:num_recommendations]

    # Map indices to movie IDs and titles
    recommended_movie_ids = [movies.iloc[i]['movieId'] for i in recommended_movie_indices]
    recommended_movies = movies.loc[movies['movieId'].isin(recommended_movie_ids), 'title']
    
    if recommended_movies.empty:
        return pd.Series(["No recommendations found. Please try another movie or adjust weights."])

    return recommended_movies

In [391]:
# Test the recommendation function 1
recommendations = recommend_hybrid(
    title="Toy Story (1995)",
    user_id=None,
    num_recommendations=10,  # Top n recommendations
    content_weight=1.0,
    collab_weight=0.0
)

# Print the recommendations
print(f"\n\nRecommendations for 'Toy Story (1995):'\n{recommendations}")

::::::: Match movie tuples before : 
:::: ('Toy Story 2 (1999)', 0.8542979675289684)
:::: ('Be Mine (2009)', 0.8311444374253919)
:::: ('A-Team, The (2010)', 0.7789465342101296)
:::: ("Ender's Game (2013)", 0.6801859968803371)
:::: ("Charlie Chan's Secret (1936)", 0.5668526438089598)
:::: ('Ghost Son (2007)', 0.5615740915932845)
:::: ("Charlie Chan's Murder Cruise (1940)", 0.5599183946694493)
:::: ('Toy, The (1982)', 0.5344726823919297)
:::: ('Starring Jerry As Himself (2023)', 0.480949864153276)
:::: ('Masallardan Geriye Kalan (2020)', 0.47761297255909585)



Movies length =>  87382
Duplicate Titles:
 6



:::::::::: Debug valid match movie tuples : 



::::::: Title to index =>  87379
::::::: Valid match movie tuples after =>  0



:::::: Length of collab weight =>  0.0
:::::: Collab weight  =>  0.0


Recommendations for 'Toy Story (1995):'
10644                tony takitani (2004)
10645        when a stranger calls (2006)
10646                       tamara (2005)
10647               

In [321]:
# Test the recommendation function
recommendations = recommend_hybrid(
    title="Toy Story (1995)",
    user_id=3,
    num_recommendations=10,  # Top n recommendations
    content_weight=0.5,
    collab_weight=0.5
)

# Print the recommendations
print(f"Recommendations for 'Toy Story (1995)':\n{recommendations}")

::::::: Match movie tuples before:
:::: ('Toy Story 2 (1999)', 0.8543294061395451)
:::: ('Toy Story 4 (2019)', 0.831265314263929)
:::: ('Toy Story 3 (2010)', 0.7789526394811229)
:::: ('Toy Story of Terror (2013)', 0.6802077184360701)
:::: ('Toy Story Toons: Small Fry (2011)', 0.5669091943311552)
:::: ('Toy Story That Time Forgot (2014)', 0.561622323525553)
:::: ('Toy Story Toons: Hawaiian Vacation (2011)', 0.5599761095784556)
:::: ('Toy, The (1982)', 0.5344121159234997)
:::: ('The New Toy (2022)', 0.48097238949521254)
:::: ('Boy Toy (2011)', 0.4776103985125679)



::::::: Title to index:
:::: Toy Story (1995)
:::: Jumanji (1995)
:::: Grumpier Old Men (1995)
:::: Waiting to Exhale (1995)
:::: Father of the Bride Part II (1995)
:::: Heat (1995)
:::: Sabrina (1995)
:::: Tom and Huck (1995)
:::: Sudden Death (1995)
:::: GoldenEye (1995)
:::: American President, The (1995)
:::: Dracula: Dead and Loving It (1995)
:::: Balto (1995)
:::: Nixon (1995)
:::: Cutthroat Island (1995)
:::: Casino (1

In [219]:
# Test the recommendation function 
recommendations = recommend_hybrid(
    title="Casino (1995)",
    user_id=None,
    num_recommendations=10,  # Top n recommendations
    content_weight=1.0,
    collab_weight=0.0
)

:::::: Length keys =>  351
Searching batch name :  batch_0
:::::: Target Index => 15, Shape => 250


In [185]:
# Test the recommendation function 3
recommendations = recommend_hybrid(
    title="Toy Story (1995)",
    user_id=4,
    num_recommendations=10,  # Top n recommendations
    content_weight=0.5,
    collab_weight=0.5
)

# Print the recommendations
print(f"Recommendations for 'Toy Story (1995)':\n{recommendations}")

Searching batch name :  batch_0
Recommendations for 'Toy Story (1995)':
8                                    Sudden Death (1995)
15                                         Casino (1995)
21                                        Copycat (1995)
22                                      Assassins (1995)
60                                 Eye for an Eye (1996)
161                                     Desperado (1995)
184                                   Nine Months (1995)
197    Umbrellas of Cherbourg, The (Parapluies de Che...
209                         Browning Version, The (1994)
212                Before the Rain (Pred dozhdot) (1994)
Name: title, dtype: object
